In [84]:
class material:
    def __init__(self,name:str,sigma_a,sigma_s,sigma_f,nu):
        
        self.name=name
        self.sigma_a=sigma_a
        self.sigma_s=sigma_s
        self.sigma_f=sigma_f
        self.nu=nu

        self.total_cross_section=self.sigma_a+self.sigma_f+sigma_s

In [85]:
#design section 
import numpy as np




X_left=0
X_right=10

Y_up=10
Y_down=0

Nx=100
Ny=Nx

dx=(X_right-X_left)/Nx
dy=(Y_up-Y_down)/Ny

X = np.linspace(dx*0.5, (X_right-X_left) - 0.5*dx,Nx)
Y = np.linspace(dx*0.5, (Y_up-Y_down) - 0.5*dx,Ny)

X,Y=np.meshgrid(X,Y)

In [86]:
active_batch=100
inactive_batch=50

total_batch=active_batch+inactive_batch

scalar_flux = np.zeros((Nx,Ny))
scalar_flux_tl = np.zeros_like (scalar_flux)


In [87]:
number_of_neutrons=10000

In [ ]:
#meshing_part 